## Pre-testing the implimenation of the magnetic probes.

Figuring out things before putting into the probe class.

In [1]:
# Create the machine, which specifies coil locations
# and equilibrium, specifying the domain to solve over
# this has to be either
# freegs.machine.MASTU(), in which case:
#tokamak = freegs.machine.MASTU()
# or
# MASTU_coils.MASTU_wpass()
import os
import numpy as np
os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"
os.environ["PROBE_PATH"] = "../machine_configs/MAST-U/magnetic_probes.pickle"

from freegsnke import machine_config
from freegsnke import build_machine
from freegsnke import faster_shape
from freegsnke import magnetic_probes 
import freegs
import pickle 

tokamak = build_machine.tokamak()


# Creates equilibrium object 
eq = freegs.Equilibrium(tokamak=tokamak,
                        #domains can be changed 
                        Rmin=0.1, Rmax=2.0,    # Radial domain
                        Zmin=-2.2, Zmax=2.2,   # Height range
                        #grid resolution can be changed
                        nx=65, ny=129, # Number of grid points
                        # psi=plasma_psi[::2,:])   
                        )  



In [2]:

current_list = eq.tokamak.getCurrents()
print(current_list)

coil_names = [name for name in current_list]
print(coil_names)

{'Solenoid': 0.0, 'PX': 0.0, 'D1': 0.0, 'D2': 0.0, 'D3': 0.0, 'Dp': 0.0, 'D5': 0.0, 'D6': 0.0, 'D7': 0.0, 'P4': 0.0, 'P5': 0.0, 'P6': 0.0, 'passive_0': 0.0, 'passive_1': 0.0, 'passive_2': 0.0, 'passive_3': 0.0, 'passive_4': 0.0, 'passive_5': 0.0, 'passive_6': 0.0, 'passive_7': 0.0, 'passive_8': 0.0, 'passive_9': 0.0, 'passive_10': 0.0, 'passive_11': 0.0, 'passive_12': 0.0, 'passive_13': 0.0, 'passive_14': 0.0, 'passive_15': 0.0, 'passive_16': 0.0, 'passive_17': 0.0, 'passive_18': 0.0, 'passive_19': 0.0, 'passive_20': 0.0, 'passive_21': 0.0, 'passive_22': 0.0, 'passive_23': 0.0, 'passive_24': 0.0, 'passive_25': 0.0, 'passive_26': 0.0, 'passive_27': 0.0, 'passive_28': 0.0, 'passive_29': 0.0, 'passive_30': 0.0, 'passive_31': 0.0, 'passive_32': 0.0, 'passive_33': 0.0, 'passive_34': 0.0, 'passive_35': 0.0, 'passive_36': 0.0, 'passive_37': 0.0, 'passive_38': 0.0, 'passive_39': 0.0, 'passive_40': 0.0, 'passive_41': 0.0, 'passive_42': 0.0, 'passive_43': 0.0, 'passive_44': 0.0, 'passive_45': 0.

In [3]:
coil_names = [el for i, el in  enumerate(eq.tokamak.getCurrents())]
print(coil_names)

['Solenoid', 'PX', 'D1', 'D2', 'D3', 'Dp', 'D5', 'D6', 'D7', 'P4', 'P5', 'P6', 'passive_0', 'passive_1', 'passive_2', 'passive_3', 'passive_4', 'passive_5', 'passive_6', 'passive_7', 'passive_8', 'passive_9', 'passive_10', 'passive_11', 'passive_12', 'passive_13', 'passive_14', 'passive_15', 'passive_16', 'passive_17', 'passive_18', 'passive_19', 'passive_20', 'passive_21', 'passive_22', 'passive_23', 'passive_24', 'passive_25', 'passive_26', 'passive_27', 'passive_28', 'passive_29', 'passive_30', 'passive_31', 'passive_32', 'passive_33', 'passive_34', 'passive_35', 'passive_36', 'passive_37', 'passive_38', 'passive_39', 'passive_40', 'passive_41', 'passive_42', 'passive_43', 'passive_44', 'passive_45', 'passive_46', 'passive_47', 'passive_48', 'passive_49', 'passive_50', 'passive_51', 'passive_52', 'passive_53', 'passive_54', 'passive_55', 'passive_56', 'passive_57', 'passive_58', 'passive_59', 'passive_60', 'passive_61', 'passive_62', 'passive_63', 'passive_64', 'passive_65', 'passiv

In [4]:
probe_path = os.environ.get("PROBE_PATH",None)

with open(probe_path, 'rb') as file:
    probe_dict = pickle.load(file) 

floops = probe_dict['flux_loops']
pickups = probe_dict['pickups']

floops_small = floops[:]
floops_pos = np.array([item['position'] for item in floops_small])
floops_pos_R = np.array([item['position'][0] for item in floops_small])
floops_pos_Z = np.array([item['position'][1] for item in floops_small])

print(floops_pos)
print(floops_pos_R)
print(floops_pos_Z)
print(floops_pos_R.shape)
# probes = magnetic_probes.probe(eq)

[[ 0.901       1.3582    ]
 [ 0.9544      1.3041999 ]
 [ 1.0445      1.2150999 ]
 [ 1.1239      1.1366    ]
 [ 1.1505      1.1987    ]
 [ 1.1555      1.1987    ]
 [ 1.2089      1.0527999 ]
 [ 0.901      -1.3628    ]
 [ 0.9544     -1.3088    ]
 [ 1.0445     -1.2198    ]
 [ 1.1239     -1.1413001 ]
 [ 1.1505     -1.2033    ]
 [ 1.1555     -1.2033    ]
 [ 1.2089     -1.0543001 ]
 [ 1.283       0.901515  ]
 [ 1.283      -0.912     ]
 [ 1.0472      1.5792    ]
 [ 1.1911      1.5732    ]
 [ 1.3724      1.5652    ]
 [ 1.5603      1.5572    ]
 [ 1.0472     -1.5833    ]
 [ 1.1911     -1.5773001 ]
 [ 1.3724     -1.5693    ]
 [ 1.5603     -1.5618    ]
 [ 1.5523      0.43915   ]
 [ 1.5523      0.38115   ]
 [ 1.5523      0.32315   ]
 [ 1.5523      0.26515   ]
 [ 1.5523     -0.27485   ]
 [ 1.5523     -0.33285   ]
 [ 1.5523     -0.39085   ]
 [ 1.5523     -0.44885   ]
 [ 0.6913123   1.7197016 ]
 [ 0.78344554  1.8108705 ]
 [ 0.87807316  1.9050441 ]
 [ 0.9505411   1.9771769 ]
 [ 1.047165    2.0733542 ]
 

In [5]:
print(floops)

[{'name': 'f_nu_01', 'position': array([0.901 , 1.3582], dtype=float32)}, {'name': 'f_nu_02', 'position': array([0.9544   , 1.3041999], dtype=float32)}, {'name': 'f_nu_03', 'position': array([1.0445   , 1.2150999], dtype=float32)}, {'name': 'f_nu_04', 'position': array([1.1239, 1.1366], dtype=float32)}, {'name': 'f_nu_a05', 'position': array([1.1505, 1.1987], dtype=float32)}, {'name': 'f_nu_b05', 'position': array([1.1555, 1.1987], dtype=float32)}, {'name': 'f_nu_06', 'position': array([1.2089   , 1.0527999], dtype=float32)}, {'name': 'f_nl_01', 'position': array([ 0.901 , -1.3628], dtype=float32)}, {'name': 'f_nl_02', 'position': array([ 0.9544, -1.3088], dtype=float32)}, {'name': 'f_nl_03', 'position': array([ 1.0445, -1.2198], dtype=float32)}, {'name': 'f_nl_04', 'position': array([ 1.1239   , -1.1413001], dtype=float32)}, {'name': 'f_nl_a05', 'position': array([ 1.1505, -1.2033], dtype=float32)}, {'name': 'f_nl_b05', 'position': array([ 1.1555, -1.2033], dtype=float32)}, {'name': '

In [6]:
eq.tokamak.coils


[('Solenoid',
  Circuit([("Solenoid", MultiCoil(R=[0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19475, 0.19

In [7]:
eq.tokamak.createPsiGreens(floops_pos_R[0],floops_pos_Z[0])
current_list = eq.tokamak.getCurrents()

greens_2darray_floops = []
for item in floops_small:
    greens_2darray_floops.append(eq.tokamak.createPsiGreens(item['position'][0],item['position'][0]))

In [8]:
# print(len(current_list))
# print(len(greens_2darray_floops[0]))
# greens_list = []
# psi_list = []
# for item in current_list:
#     greens_list.append(greens_2darray_floops[0][item])

# for item in greens_list:
#     psi_list.append()

# print(len(current_list))
# print(len(greens_list))



699
699


TypeError: list.append() takes exactly one argument (0 given)

In [9]:
greens_2darray_floops
print(len(greens_2darray_floops))
arr =  np.array([1,2,3,4,5])
newarr = arr[:,np.newaxis]

mult = []
for el in greens_2darray_floops:
    # mult.append(greens_2darray_floops[el]*5)
    print(el)



102
{'Solenoid': {'Solenoid': 1.886405506847333e-06}, 'PX': {'PX1': 7.437589640948215e-07, 'PX2': 5.2970956804634955e-08}, 'D1': {'D11': 9.078446639528288e-07, 'D12': 7.173445959432965e-08}, 'D2': {'D21': 8.945827843728602e-07, 'D22': 8.201254820776442e-08}, 'D3': {'D31': 1.0511760781850191e-06, 'D32': 1.2586685725833182e-07}, 'Dp': {'Dp1': 2.8411316511197314e-06, 'Dp2': 2.5460650510549144e-07}, 'D5': {'D51': 2.387401126116255e-06, 'D52': 5.801514163708413e-07}, 'D6': {'D61': 3.4336977490051957e-06, 'D62': 4.3813263488982605e-07}, 'D7': {'D71': 3.2665402752152485e-06, 'D72': 5.49686689879127e-07}, 'P4': {'P41': 4.3893769542180414e-06, 'P42': 7.566232234711282e-07}, 'P5': {'P51': 3.1798090958222853e-06, 'P52': 1.666095656841224e-06}, 'P6': {'P61': 2.408950414728337e-06, 'P62': 3.329031835721573e-07}, 'passive_0': 1.0794377492548673e-07, 'passive_1': 9.967948530692418e-08, 'passive_2': 8.981628191793057e-08, 'passive_3': 7.893190509290253e-08, 'passive_4': 5.935900541802596e-08, 'passive

In [11]:
print(pickups)
pickup_orientation = [el['orientation_vector'] for el in pickups]
pickup_position = [el['position'] for el in pickups]
print(pickup_orientation)

pickup_orientation[1][0]

[{'name': 'b_c1_p01', 'position': array([2.76900e-01, 3.00000e+02, 1.26203e+00], dtype=float32), 'orientation': 'PARALLEL', 'orientation_vector': array([0., 0., 1.], dtype=float32)}, {'name': 'b_c1_t02', 'position': array([2.7689108e-01, 2.9999680e+02, 1.2245095e+00], dtype=float32), 'orientation': 'TOROIDAL', 'orientation_vector': array([0., 1., 0.], dtype=float32)}, {'name': 'b_c1_p03', 'position': array([2.7689999e-01, 3.0000000e+02, 1.1870301e+00], dtype=float32), 'orientation': 'PARALLEL', 'orientation_vector': array([0., 0., 1.], dtype=float32)}, {'name': 'b_c1_t04', 'position': array([2.7689108e-01, 2.9999680e+02, 1.1495094e+00], dtype=float32), 'orientation': 'TOROIDAL', 'orientation_vector': array([0., 1., 0.], dtype=float32)}, {'name': 'b_c1_p05', 'position': array([2.76900e-01, 3.00000e+02, 1.11203e+00], dtype=float32), 'orientation': 'PARALLEL', 'orientation_vector': array([0., 0., 1.], dtype=float32)}, {'name': 'b_c1_t06', 'position': array([2.7689108e-01, 2.9999680e+02, 1

0.0

In [ ]:

def pickup_value(eq):
        """ 
        Computes value of B.n at each pickup probe postition.
        Uses Br,Btor,Bz methods defined already for equilibrium object. 
        # i'm asumming that these already do the appropriate sums over the coils/grid/plasma etc.
        """
        BdotN = []
        for i, el in enumerate(pickups):
            # only need r, z positions as B = B(r,z)
            R,Z = pickup_position[i][0], pickup_position[i][2]
            orientation_vec = pickup_orientation[i]
            Bvec = [eq.Br(R,Z),eq.Btor(R,Z),eq.Bz(R,Z)]

            BdotN.append(np.dot(Bvec,orientation_vec))
        
        return BdotN



In [ ]:
from freegsnke import machine_config
coildict = machine_config.coils_dict

from freegs.gradshafranov import Greens ,GreensBr, GreensBz

 

In [ ]:
print(coildict.keys())
coildict['D1']['coords'][0]
# coildict['D1']['coords'][0][:,np.newaxis]



dict_keys(['Solenoid', 'PX', 'D1', 'D2', 'D3', 'Dp', 'D5', 'D6', 'D7', 'P4', 'P5', 'P6', 'passive_0', 'passive_1', 'passive_2', 'passive_3', 'passive_4', 'passive_5', 'passive_6', 'passive_7', 'passive_8', 'passive_9', 'passive_10', 'passive_11', 'passive_12', 'passive_13', 'passive_14', 'passive_15', 'passive_16', 'passive_17', 'passive_18', 'passive_19', 'passive_20', 'passive_21', 'passive_22', 'passive_23', 'passive_24', 'passive_25', 'passive_26', 'passive_27', 'passive_28', 'passive_29', 'passive_30', 'passive_31', 'passive_32', 'passive_33', 'passive_34', 'passive_35', 'passive_36', 'passive_37', 'passive_38', 'passive_39', 'passive_40', 'passive_41', 'passive_42', 'passive_43', 'passive_44', 'passive_45', 'passive_46', 'passive_47', 'passive_48', 'passive_49', 'passive_50', 'passive_51', 'passive_52', 'passive_53', 'passive_54', 'passive_55', 'passive_56', 'passive_57', 'passive_58', 'passive_59', 'passive_60', 'passive_61', 'passive_62', 'passive_63', 'passive_64', 'passive_65

array([0.35275, 0.36745, 0.38215, 0.39685, 0.41155, 0.42625, 0.35275,
       0.36745, 0.38215, 0.39685, 0.41155, 0.42625, 0.35275, 0.36745,
       0.38215, 0.39685, 0.41155, 0.42625, 0.35275, 0.36745, 0.38215,
       0.39685, 0.41155, 0.42625, 0.35275, 0.36745, 0.38215, 0.39685,
       0.41155, 0.42625, 0.35275, 0.36745, 0.38215, 0.39685, 0.41155,
       0.35275, 0.36745, 0.38215, 0.39685, 0.41155, 0.42625, 0.35275,
       0.36745, 0.38215, 0.39685, 0.41155, 0.42625, 0.35275, 0.36745,
       0.38215, 0.39685, 0.41155, 0.42625, 0.35275, 0.36745, 0.38215,
       0.39685, 0.41155, 0.42625, 0.35275, 0.36745, 0.38215, 0.39685,
       0.41155, 0.42625, 0.35275, 0.36745, 0.38215, 0.39685, 0.41155])

In [ ]:
greens_array = greens_coils()
greens_array


TypeError: list indices must be integers or slices, not tuple

In [12]:
# Numpy things... multiplyting arrays - matrix mult, 
list_1 = np.array([1,2,3,4,5,6,7,8,9])
list_2 = np.array([5,4,3])
list_3 = np.array([6,7,3])
# multiplication is element wise
print(list_2*list_3)
print(list_2[:,np.newaxis])
print(list_2[np.newaxis,:])

print('list 1 = {}, \nlist 2 = {}'.format(list_1,list_2))
print('scalar multiplication \n',list_1 *3 )
print('outerproduct v1 \n',list_1 *list_2[:,np.newaxis])
print('outerproduct v2 \n',list_1[:,np.newaxis]* list_2)
print('outerproduct v3 \n',list_1[np.newaxis,:]*list_2[:,np.newaxis])
print('sum v1 axis 0 \n', np.sum(list_1 *list_2[:,np.newaxis],axis = 0))
print('sum v1 axis 1\n ', np.sum(list_1 *list_2[:,np.newaxis],axis = 1))
print('sum v2 axis = 0\n', np.sum(list_1[:,np.newaxis]* list_2,axis = 0))
print('sum v2 axis = 1\n', np.sum(list_1[:,np.newaxis]* list_2,axis = 1))

[30 28  9]
[[5]
 [4]
 [3]]
[[5 4 3]]
list 1 = [1 2 3 4 5 6 7 8 9], 
list 2 = [5 4 3]
scalar multiplication 
 [ 3  6  9 12 15 18 21 24 27]
outerproduct v1 
 [[ 5 10 15 20 25 30 35 40 45]
 [ 4  8 12 16 20 24 28 32 36]
 [ 3  6  9 12 15 18 21 24 27]]
outerproduct v2 
 [[ 5  4  3]
 [10  8  6]
 [15 12  9]
 [20 16 12]
 [25 20 15]
 [30 24 18]
 [35 28 21]
 [40 32 24]
 [45 36 27]]
outerproduct v3 
 [[ 5 10 15 20 25 30 35 40 45]
 [ 4  8 12 16 20 24 28 32 36]
 [ 3  6  9 12 15 18 21 24 27]]
sum v1 axis 0 
 [ 12  24  36  48  60  72  84  96 108]
sum v1 axis 1
  [225 180 135]
sum v2 axis = 0
 [225 180 135]
sum v2 axis = 1
 [ 12  24  36  48  60  72  84  96 108]


In [25]:
# joining numpy arrays - np version of append...
vstack = np.vstack((list_2,list_2))
hstack = np.hstack((list_2,list_3))
hstack2 = np.hstack((list_1,list_2))
concat = np.concatenate((list_1,list_2),axis= 0)
print(vstack)
print(hstack)
print(hstack2)
print(concat)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
print(Greens(1,1,list_3,list_3))
list_4 = []
for i , el in enumerate(list_1):
    list_4.append(Greens(el,el,list_2[:,np.newaxis],list_2[:,np.newaxis]))
print(list_4)
print(np.sum(list_4, axis = 0))


[2.35838279e-08 1.95458468e-08 5.92731442e-08]
[array([[2.96512043e-08],
       [3.97100228e-08],
       [5.92731442e-08]]), array([[1.55535116e-07],
       [2.22413451e-07],
       [3.77085833e-07]]), array([[4.60483273e-07],
       [7.32387643e-07],
       [6.53953187e-06]]), array([[1.14910888e-06],
       [8.71937583e-06],
       [7.32387643e-07]]), array([[1.08992198e-05],
       [1.14910888e-06],
       [4.60483273e-07]]), array([[1.61256473e-06],
       [7.54171666e-07],
       [3.33620176e-07]]), array([[1.09155167e-06],
       [5.61281884e-07],
       [2.59705577e-07]]), array([[8.29350118e-07],
       [4.44826902e-07],
       [2.11531873e-07]]), array([[6.67030139e-07],
       [3.66821306e-07],
       [1.77819433e-07]])]
[[1.68944949e-05]
 [1.29900976e-05]
 [9.15143882e-06]]


In [ ]:
def fun(x,y):
    return x + y

In [ ]:
y1 = fun(list_1,4)
y2 = fun(list_1,list_2[:,np.newaxis])
print(y2)
y2.shape
y3 = Greens(list_1,list_1,list_2[:,np.newaxis],list_3[:,np.newaxis])
print(y3)
print(y3.shape)



[[ 6  7  8  9 10 11 12 13 14]
 [ 5  6  7  8  9 10 11 12 13]
 [ 4  5  6  7  8  9 10 11 12]]
[[2.19629872e-08 1.15175126e-07 3.37145448e-07 7.91914759e-07
  1.71398415e-06 1.97348562e-06 1.33068783e-06 9.86211477e-07
  7.78390994e-07]
 [1.31685969e-08 7.09144114e-08 2.08419464e-07 4.60685874e-07
  7.91914759e-07 9.42036156e-07 8.29733078e-07 6.74139700e-07
  5.49173322e-07]
 [5.92731442e-08 3.77085833e-07 6.53953187e-06 7.32387643e-07
  4.60483273e-07 3.33620176e-07 2.59705577e-07 2.11531873e-07
  1.77819433e-07]]
(3, 9)


In [ ]:
div = 1 / list_1
div 

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
       0.16666667, 0.14285714, 0.125     , 0.11111111])

In [ ]:
# print(coildict[key]['coords'])
# print(floops_pos_R[:,np.newaxis])

def greens_coils():
    greens_array = []
    for key in coildict.keys():
            pol = coildict[key]['polarity'][:,np.newaxis]
            mul = coildict[key]['multiplier'][:,np.newaxis]
            greens_array.append(np.sum(Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis]),axis= 1))
            # greens_array.append(Greens(coildict[key]['coords'][0][:,np.newaxis],
            #                        coildict[key]['coords'][1][:,np.newaxis],
            #                        floops_pos_R,
            #                        floops_pos_Z))
    return greens_array
def greens_coils2():
    greens_array = []
    for key in coildict.keys():
            greens_array.append(Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis]))
            
    return greens_array

def greens_coils3(key):
    pol = coildict[key]['polarity'][np.newaxis,:]
    mul = coildict[key]['multiplier'][np.newaxis,:]
    greens = Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis])
    greens *= pol
    greens *= mul
    return greens
def greens_4():
    arr = []
    for key in coildict.keys():
            a = np.sum(greens_coils3(key),axis =1)
            arr.append(a)
    return np.array(arr) 

In [ ]:
a = greens_coils3('Solenoid')
print(a)
print(a.shape)
print(np.sum(a,axis=1))
b = greens_4()
print(b)
print(b.shape)
c = np.sum(b,axis=0)
print(c)


[[3.31152563e-10 3.34185644e-10 3.37254973e-10 ... 1.24331536e-08
  1.23389640e-08 1.22416650e-08]
 [3.84635718e-10 3.88174822e-10 3.91756249e-10 ... 1.13469185e-08
  1.12533850e-08 1.11577568e-08]
 [4.86206544e-10 4.90701303e-10 4.95249650e-10 ... 9.81288491e-09
  9.72658420e-09 9.63922483e-09]
 [5.88594067e-10 5.94042679e-10 5.99555810e-10 ... 8.70861943e-09
  8.63066635e-09 8.55216003e-09]
 [5.76300315e-10 5.81514690e-10 5.86789408e-10 ... 9.03230729e-09
  8.96346846e-09 8.89372256e-09]]
(5, 324)
[1.46696772e-06 1.50513829e-06 1.55210273e-06 1.58096176e-06
 1.51681726e-06]
[[1.46696772e-06 1.50513829e-06 1.55210273e-06 1.58096176e-06
  1.51681726e-06]
 [8.99604225e-07 8.73905103e-07 8.18741558e-07 7.67268810e-07
  7.55452792e-07]
 [1.82597873e-06 1.65987941e-06 1.43965271e-06 1.29120295e-06
  1.34458096e-06]
 ...
 [9.58598170e-08 1.12201630e-07 1.44262444e-07 1.78170166e-07
  1.75932921e-07]
 [8.69116145e-08 1.02004987e-07 1.31595150e-07 1.62808849e-07
  1.58189439e-07]
 [9.57907434

In [ ]:
pol = coildict['Solenoid']['polarity'][np.newaxis,:]
mul = coildict['Solenoid']['multiplier'][:,np.newaxis]

coiltest = greens_coils3('Solenoid')
print(coiltest.shape)
print(len(pol))
print(pol.shape)
coiltestnew = coiltest*pol
print(coiltest)
print(coiltestnew.shape)


(5, 324)
1
(1, 324)
[[3.31152563e-10 3.34185644e-10 3.37254973e-10 ... 1.24331536e-08
  1.23389640e-08 1.22416650e-08]
 [3.84635718e-10 3.88174822e-10 3.91756249e-10 ... 1.13469185e-08
  1.12533850e-08 1.11577568e-08]
 [4.86206544e-10 4.90701303e-10 4.95249650e-10 ... 9.81288491e-09
  9.72658420e-09 9.63922483e-09]
 [5.88594067e-10 5.94042679e-10 5.99555810e-10 ... 8.70861943e-09
  8.63066635e-09 8.55216003e-09]
 [5.76300315e-10 5.81514690e-10 5.86789408e-10 ... 9.03230729e-09
  8.96346846e-09 8.89372256e-09]]
(5, 324)


In [ ]:
print(len(coildict['D1']['coords'][0]))
print(len(coildict['D1']['multiplier']))



70
70


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
array = np.array([[1,2,3],[4,5,6]])
array2  =np.array([1,-1,0])
array*array2

array([[ 1, -2,  0],
       [ 4, -5,  0]])

In [ ]:
coil_names

array_of_coil_currents = np.array([eq.tokamak[label].current for label in current_list])
greens_all_coils = greens_4()
print(len(array_of_coil_currents))
print(len(greens_all_coils))

psi_all_coils = greens_all_coils * array_of_coil_currents[:,np.newaxis]

psi_sum_all_coils = np.sum(psi_all_coils,axis = 0)
print(psi_sum_all_coils)

699
699
[0. 0. 0. 0. 0.]


In [ ]:
nx ,ny = 11,6
Rmin, Rmax = 0,10
Zmin, Zmax = 5 ,10

R_1D = np.linspace(Rmin, Rmax, nx)
Z_1D = np.linspace(Zmin, Zmax, ny)
R, Z = np.meshgrid(R_1D, Z_1D, indexing="ij")

In [ ]:
R[np.newaxis , :,:]

array([[[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  5.,  5.,  5.],
        [ 6.,  6.,  6.,  6.,  6.,  6.],
        [ 7.,  7.,  7.,  7.,  7.,  7.],
        [ 8.,  8.,  8.,  8.,  8.,  8.],
        [ 9.,  9.,  9.,  9.,  9.,  9.],
        [10., 10., 10., 10., 10., 10.]]])

In [ ]:
X = Greens(R[:,:,np.newaxis],Z[:,:,np.newaxis],floops_pos_R[np.newaxis,np.newaxis,:],floops_pos_Z[np.newaxis,np.newaxis,:])
Y = np.sum(X,axis = (1,2))
print(X.shape)
print(Y.shape)

(11, 6, 5)
(11,)


In [ ]:
eq._profiles.jtor

AttributeError: 'Equilibrium' object has no attribute '_profiles'

In [ ]:
abc = [[1,2],[3,4],[5,6],[7,8]]
abc[0][0]

1